# Correlation Matrix Baseline

In [1]:
%load_ext autoreload

In [2]:
import os
import parkinson

import pandas as pd
import networkx as nx
import numpy as np

from fastdtw import fastdtw
from tqdm import tqdm
from node2vec import Node2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.spatial.distance import euclidean

In [ ]:
control_data = parkinson.utils.data.batch_read('../data/Controls_columns')
parkinson_data = parkinson.utils.data.batch_read('../data/PDs_columns')

100%|██████████| 66/66 [00:02<00:00, 23.28it/s]


Os atlas possíveis são AAL3, Shen_268 e atlas. Cada atlas particiona o cérebro de maneiras diferentes. Devemos selecionar as colunas correspondentes para aplicar o processamento

In [ ]:
control_atlas_data = parkinson.utils.data.select_atlas_columns(control_data, 'atlas')
parkinson_atlas_data = parkinson.utils.data.select_atlas_columns(parkinson_data, 'atlas')

## Create Graphs

Demora muito para calcular a rede funcional com DTW

### Teste com a Triangular Superior

In [ ]:
parkinson_correlation_matrix = [parkinson.utils.graph.compute_correlation_matrix(time_series) for time_series in parkinson_atlas_data]
control_correlation_matrix = [parkinson.utils.graph.compute_correlation_matrix(time_series) for time_series in control_atlas_data]

X = parkinson.utils.data.concatenate_data(parkinson_correlation_matrix, control_correlation_matrix)
y = parkinson.utils.data.concatenate_data([1 for _ in range(len(parkinson_data))], [0 for _ in range(len(control_data))])

#### Removendo pacientes com erros

In [8]:
X, y = parkinson.utils.data.filter_data(X, y)

#### Train/Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
params = {
    'alpha': np.arange(1e-4, 1e-2, 1e-3),
    'learning_rate': ['constant', 'adaptive'],
}

#### Training

In [11]:
model = MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100))
hyperparam_optimization = RandomizedSearchCV(model, params, random_state=1)
search = hyperparam_optimization.fit(X_train, y_train)
search.best_params_

{'learning_rate': 'adaptive', 'alpha': 0.0001}

In [12]:
optimized_model = MLPClassifier(early_stopping=True, **search.best_params_).fit(X_train, y_train)
optimized_model.score(X_test, y_test)

0.6666666666666666

In [13]:
optimized_model.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1])